In [70]:
import mbuild as mb
import foyer
from mbuild.examples import Ethane
import parmed
import subprocess


In [71]:
#cmpd = Ethane()
#cmpd.name = 'Eth'
cmpd = mb.Compound()
part = mb.Particle(name='Ar')
cmpd.add(part)
cmpd.name='LJ'

box = mb.fill_box(cmpd, n_compounds=10, box=[10,10,10])
ff = foyer.Forcefield(forcefield_files='lj.xml')
non_param = box.to_parmed(residues='LJ')
param = ff.apply(non_param)


/Users/ayang41/anaconda3/envs/py36/lib/python3.6/site-packages/parmed/openmm/topsystem.py:238: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)


In [72]:
def write_gomc_input(filename, parm_structure, coords='coords.pdb', 
                     structure='structure.psf', parameters='parameters.par',
                    output='output'):
    with open(filename, 'w') as f:
        f.write(""" 
###########################################################################
#  ========-------------------- INPUT --------------------------===========
############################################################################

#########################
# enable, step
#########################
Restart     false


####################################
# kind (RESTART, RANDOM, INTSEED)
####################################
PRNG        RANDOM

####################################
# FORCE FIELD
####################################
ParaTypeCHARMM   true
ParaTypeEXOTIC   false
Parameters      {parameters}

####################################
# INPUT PDB FILES
####################################
Coordinates 0    {coords}

####################################
# INPUT PSF FILES
####################################
Structure 0      {structure}



############################################################################
#  =======--------------------- SYSTEM --------------------------===========
############################################################################

##################################
# GEMC TYPE (DEFULT IS NVT_GEMC)
##################################


#############################
# SIMULATION CONDITION
#############################
Temperature     300.00
Potential       VDW
LRC     true
Rcut        10
Exclude     1-4

#############################
# ELECTROSTATIC
#############################
ElectroStatic   false
Ewald           false
PressureCalc  true  1000

################################
# STEPS
################################
RunSteps           1000000
EqSteps        500000
AdjSteps       1000

################################
# MOVE FREQUENCY
################################
DisFreq               0.60
RotFreq           0.10
RegrowthFreq          0.30


################################
# BOX DIMENSION #, X, Y, Z
################################
CellBasisVector1 0  {box_x}  0.00    0.00
CellBasisVector2 0  0.00    {box_y}  0.00
CellBasisVector3 0  0.00    0.00    {box_z}


##############################
# CBMC TRIALS
##############################
CBMC_First   10
CBMC_Nth     8
CBMC_Ang     100
CBMC_Dih     20

####################################
#          Mol. Name     Chem. Pot.
####################################
############################################################################
#  =======-------------------- OUTPUT --------------------------===========
############################################################################

##########################
# statistics filename add
##########################
OutputName  {output}

#####################################
# enable, frequency
#####################################
CoordinatesFreq    true   1000000
RestartFreq        true   1000000
ConsoleFreq        true   100000
BlockAverageFreq   true   100000
HistogramFreq      false  100000


################################
# OutHistSettings
################################


##################################
# enable: blk avg., fluct.
##################################
OutEnergy         true    true
OutPressure       true    true
OutMolNum         true    true
OutDensity        true    true
""".format(structure=structure, coords=coords, parameters=parameters, output=output,
          box_x=parm_structure.box[0], box_y=parm_structure.box[1], box_z=parm_structure.box[2]))

In [73]:
param.save('coords.pdb', overwrite=True)
param.save('structure.psf', overwrite=True)
paramset = parmed.charmm.CharmmParameterSet.from_structure(param)
paramset.write(par='parameters.par')
write_gomc_input('in.conf', param, coords='coords.pdb', structure='structure.psf', parameters='parameters.par')

In [74]:
param.atoms[0].type

'AR'